In [ ]:
# %pip install langchain
# %pip install huggingface_hub
# %pip install sentence_transformers
# %pip install faiss-cpu
# %pip install unstructured
# %pip install chromadb
# %pip install Cython
# %pip install tiktoken
# %pip install unstructured[local-inference]
# %pip install qdrant-client

In [1]:
# import os
# import requests
# import faiss
# import torch

# from langchain.embeddings import HuggingFaceEmbeddings #for using HugginFace models
# # Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
# from langchain.vectorstores import FAISS  #facebook vectorizationfrom langchain.chains.question_answering import load_qa_chain
# from langchain.chains.question_answering import load_qa_chain
# from langchain.document_loaders import DirectoryLoader
# from langchain.document_loaders import TextLoader
# from langchain.embeddings import HuggingFaceEmbeddings

# from LangChain_chatbot_util import read_api_key

# from langchain.text_splitter import CharacterTextSplitter

# os.environ["HUGGINGFACEHUB_API_TOKEN"] = read_api_key()

# from langchain import HuggingFacePipeline
# from langchain import PromptTemplate,  LLMChain

# from langchain.memory import ConversationBufferMemory
# from langchain import LLMChain, PromptTemplate

# from transformers import AutoTokenizer, AutoModel

# import pandas as pd

from LangChain_chatbot_util import *

2023-11-18 12:18:23.468466: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-18 12:18:23.508978: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
LLMFactory.initiate_model()

importing model and tokenizer from HuggingFace


/home/milleraa/.local/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:674: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/milleraa/.local/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
output = llm_response("My child might have autism. He likes trucks")

Created a chunk of size 565, which is longer than the specified 500
Created a chunk of size 534, which is longer than the specified 500


[INST] <<SYS>>
You are a helpful assistant, you always only answer for the 
        assistant then you stop. Read the chat history and reference the context provided 
        to better be able to respond to the prompt
<</SYS>>

 
Context:

 Delays in communicating with others including speech and gestures

Not responding to one’s own name

Getting upset over minor changes in routine

Avoiding many, if not most, forms of physical contact

Showing little interest in objects or people

The American Academy of Pediatrics (AAP) suggests that autism screenings should be part of standard 18 and 24

month well

 Examples of restricted or repetitive behaviors and interests related to ASD can include

Lines up toys or other objects and gets upset when order is changed

Repeats words or phrases over and over (called echolalia)

Plays with toys the same way every time

Is focused on parts of objects (for example, wheels)

Gets upset by minor changes

Has obsessive interests

Must follow certain ro

In [6]:
print(output)

  Hello! I'm here to help you understand more about autism and how it may affect your child. It's important to keep in mind that every individual with autism is unique and may exhibit a different combination of signs and symptoms.

Based on the information you provided, it's possible that your child may be exhibiting some common traits associated with autism, such as a preference for routine and a limited interest in social interactions. It's also possible that your child may be experiencing delays in communication and language skills, as well as difficulty responding to their own name.

It's important to consult with a medical professional to determine if your child is exhibiting signs of autism. The American Academy of Pediatrics recommends that autism screenings be part of standard 18 and 24-month well-child visits. Early identification and intervention can make a significant difference in the lives of individuals with autism, and can help them reach their full potential.

In the me

In [ ]:
loader = DirectoryLoader('./contexts', glob="./*.txt")
 
docs = loader.load()

In [ ]:
print(docs[0].page_content)

In [ ]:
text_splitter = CharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=0
)

docs_split = text_splitter.split_documents(docs)
docs_split

In [ ]:
# Embeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
db = FAISS.from_documents(docs_split, embeddings)
query = "My teenager had a screening for autism. Where can I go for depression therapy services? "
docs_similar = db.similarity_search_with_score(query)

doc0, sim = docs_similar[0]
sim
# for i in range(4):
#     print("\n")
#     print(docs[i].page_content)

In [ ]:
def get_similarity(query, doc):
    db = FAISS.from_documents(doc, HuggingFaceEmbeddings())
    db_docs = db.similarity_search_with_score(query)
    simularity_list = []
    doc_list = []
    
    for item in db_docs:
        simularity, doc = item
        simularity_list.append(simularity)
        doc_list.append(doc)
    return simularity_list, doc_list

In [ ]:
sim, doc = get_similarity("My child keeps eating legos. Is he Autstic", docs_split)

In [ ]:
df = pd.DataFrame({"Document":doc, "Simularity": sim})
df.head()

In [ ]:
prompt = "My child keeps eating legos. Is he Autstic"
similarity_scores, indices = get_similarity(prompt, db)

# Print the similarity scores and corresponding document indices
print("Similarity Scores:", similarity_scores)
print("Document Indices:", indices)

In [ ]:
index = faiss.IndexFlatL2()